COS80024

DATA SCIENCE PROJECT 1

PROJECT 4: MOVIE RECOMMENDATION SYSTEM

# Task S3.3 Content-Based Filtering

This task aims to develop, select, train and tune parameters for a content-based filter.

Code Contributed by - Asif Rana, Student ID - 102866893

In [1]:
# Importing all the necessary libraries
import numpy as np
import pandas as pd
# Importing all the necessary libraries
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error
from sklearn import metrics
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.feature_extraction.text import TfidfVectorizer
import sklearn.metrics as sm
from sklearn.metrics import mean_absolute_error
import time

In [2]:
# Importing the csv data files 
movies_metadata = pd.read_csv('metadata_integrated.csv',error_bad_lines=False, warn_bad_lines=True)
train_df = pd.read_csv('train_df_10.csv',error_bad_lines=False, warn_bad_lines=True)
test_df = pd.read_csv('test_df_10.csv',error_bad_lines=False, warn_bad_lines=True)

# Creating a new attribute by concating the title with the movieId to make the tiles unique
train_df['concat_title'] = train_df['title'].str.lower().str.replace(' ','') + train_df['movieId'].astype(str)
test_df['concat_title'] = test_df['title'].str.lower().str.replace(' ','') + test_df['movieId'].astype(str)


### Data Transformations

In [3]:
content_recom_df = movies_metadata[['movieId', 'overview','genres','title', 'production_companies', 'collectionName']]

In [4]:
content_recom_df['genres'] = content_recom_df['genres'].str.replace(',',' ').str.lower()
content_recom_df['concat_title'] = content_recom_df['title'].str.lower().str.replace(' ','') + content_recom_df['movieId'].astype(str)
content_recom_df['overview'] = content_recom_df['overview'].str.lower()
content_recom_df['collectionName'] = content_recom_df['collectionName'].str.replace(' ','').str.lower()
content_recom_df['production_companies'] = content_recom_df['production_companies'].str.replace(' ','').str.lower().str.replace(',',' ')

C:\Users\user\anaconda3New\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\user\anaconda3New\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\user\anaconda3New\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cave

##  Model- using title, genre, overview, collectionName and production_company

In [5]:
# Create the corpus of data from the metadata of movies
content_recom_df['corpus'] = content_recom_df['title'].str.lower() + ' ' + content_recom_df['genres'] + ' ' + content_recom_df['overview'] + ' ' + content_recom_df['collectionName'] + ' ' + content_recom_df['production_companies'] 
content_recom_df['corpus'] = content_recom_df['corpus'].str.replace('[^\w\s]','') 
content_recom_df['corpus'] = content_recom_df['corpus'].str.replace('unknown','') 

C:\Users\user\anaconda3New\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\user\anaconda3New\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\user\anaconda3New\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexe

In [6]:
# Reset index of your df and construct reverse mapping again
content_recom_df = content_recom_df.reset_index()
indices = pd.Series(content_recom_df.index, index=content_recom_df['concat_title'])

In [7]:
#Define a TF-IDF Vectorizer Object. Removing all english stopwords
tfidf = TfidfVectorizer(stop_words='english')


#Constructing the required TF-IDF matrix by applying the fit_transform method on the overview feature
tfidf_matrix = tfidf.fit_transform(content_recom_df['corpus'])

#Output the shape of tfidf_matrix
tfidf_matrix.shape

(9042, 43175)

In [8]:
# Import linear_kernel to compute the dot product
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix
cosine_sim_tfidf = linear_kernel(tfidf_matrix, tfidf_matrix)

In [9]:
# Getting the unique list of useres in the test dataset
users= test_df.userId.unique()

In [10]:
# This is the main function which predicts the ratings for the movies
def predit_content_rating(cosine_sim):
    users= test_df.userId.unique()
    for k in range(len(users)):
        for i in range(len(test_df.loc[(test_df['userId'] == users[k])])):
            temp_train_df = train_df.loc[(train_df['userId'] == users[k])]
            temp_test_df = test_df.loc[(test_df['userId'] == users[k])]
            sum_weight = 0
            rating_weight = 0
            prediction = 0
            title_idx = indices[temp_test_df.iloc[i].concat_title]
            ## Changed this line to cosine_sim2
            sim_scores = list(enumerate(cosine_sim[title_idx]))
            for j in range(len(temp_train_df)):
                idx = indices[temp_train_df.iloc[j].concat_title]
                weight = sim_scores[idx][1]
                rating_weight += (weight * (temp_train_df.iloc[j].rating))
                sum_weight += weight
            prediction = (rating_weight/sum_weight)
            test_df.loc[(test_df['userId'] == users[k]) & (test_df['title'] == temp_test_df.iloc[i].title),'predict'] = prediction

##  Corpus using - title, genre, overview, collectionName, production_companies

## Performance Evaluation using TF-IDF

In [11]:
# Model predicting the ratings of the movies using TF-IDF as feature engineering
predit_content_rating(cosine_sim_tfidf)

In [35]:
global_mean_rating = np.average(train_df.rating.values)

In [36]:
test_df = test_df.fillna(global_mean_rating)

In [12]:
# The actual rating of the movies by the user
y_test = test_df['rating'].values
# The predicted rating of the movies
y_test_pred = test_df['predict'].values

In [13]:
# Root Mean Square Error of the model
rms = round(sqrt(mean_squared_error(y_test, y_test_pred )), 2)
print('Root Mean Square Error:',rms)

Root Mean Square Error: 0.98


In [14]:
# Mean Absolute Error of the model
mse = round(mean_absolute_error(y_test, y_test_pred), 2)
print('Mean Absolute Error:',mse)

Mean Absolute Error: 0.75


##  Corpus using - title, genre, overview

In [15]:
content_recom_df['corpus'] =  content_recom_df['genres'] + ' ' + content_recom_df['overview']  
content_recom_df['corpus'] = content_recom_df['corpus'].str.replace('[^\w\s]','') 
content_recom_df['corpus'] = content_recom_df['corpus'].str.replace('unknown','') 

In [16]:
#Define a TF-IDF Vectorizer Object. Removing all english stopwords
tfidf = TfidfVectorizer(stop_words='english')


#Constructing the required TF-IDF matrix by applying the fit_transform method on the overview feature
tfidf_matrix = tfidf.fit_transform(content_recom_df['corpus'])

#Output the shape of tfidf_matrix
tfidf_matrix.shape

(9042, 33717)

In [17]:
# Import linear_kernel to compute the dot product
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix
cosine_sim_tfidf = linear_kernel(tfidf_matrix, tfidf_matrix)

In [18]:
# Model predicting the ratings of the movies using TF-IDF as feature engineering
predit_content_rating(cosine_sim_tfidf)

C:\Users\user\anaconda3New\lib\site-packages\ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in double_scalars


### Performance Evaluation on the test data

In [19]:
global_mean_rating = np.average(train_df.rating.values)

In [20]:
global_mean_rating

3.5356324435981024

In [21]:
test_df = test_df.fillna(global_mean_rating)

In [22]:
# The actual rating of the movies by the user
y_test = test_df['rating'].values
# The predicted rating of the movies
y_test_pred = test_df['predict'].values

In [23]:
# Root Mean Square Error of the model
rms = round(sqrt(mean_squared_error(y_test, y_test_pred )), 2)
print('Root Mean Square Error:',rms)

Root Mean Square Error: 0.98


In [24]:
# Mean Absolute Error of the model
mse = round(mean_absolute_error(y_test, y_test_pred), 2)
print('Mean Absolute Error:',mse)

Mean Absolute Error: 0.75


### The two model gives the same accuracy so I have decided to use the model which uses overview and genre features

In [25]:
#Exporting the predicted values
test_df.to_csv("predicted_content_based_test_df.csv", index=False)

### Predicting the ratings in the train dataset

In [26]:
train_df = train_df[['userId','movieId','rating']]
test_df1 = train_df[['userId','movieId','rating']]

In [27]:
# Reset index of your df and construct reverse mapping again
content_recom_df = content_recom_df.reset_index()
indices = pd.Series(content_recom_df.index, index=content_recom_df['movieId'])

In [28]:
start = time.time()
users = test_df1.userId.unique()
k = 36 # Setting the value for the most similar movies number
start = time.time()
for i in range(len(users)):
    start1 = time.time()
    rated_movies = train_df.loc[(train_df['userId'] == users[i])].movieId.values
    unrated_movies = test_df1.loc[(test_df1['userId'] == users[i])].movieId.values
    for movie in rated_movies:
        idx = indices[movie]
        sim_scores = list(enumerate(cosine_sim_tfidf[idx]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        sim_scores = sim_scores[1:k]
        movie_indices = [i[0] for i in sim_scores]
        movieIds = movies_metadata['movieId'].iloc[movie_indices].values    
        unrated_movies = list(set(movieIds).intersection(unrated_movies))
        if len(unrated_movies) > 0:
            for movie in unrated_movies:
                idx = indices[movie]
                sim_scores = list(enumerate(cosine_sim_tfidf[idx]))
                sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
                sim_scores = sim_scores[1:k]
                movieIds = movies_metadata['movieId'].iloc[movie_indices].values  
                movie_indices = [i[0] for i in sim_scores]
                
                movieIds = movies_metadata['movieId'].iloc[movie_indices].values  
                unrated_movies = list(set(movieIds).intersection(rated_movies))
                if len(unrated_movies)> 0:
                    weight_movies = [i[1] for i in sim_scores]
                    temp = pd.DataFrame()
                    temp2 = pd.DataFrame()
                    merged_rating_df = pd.DataFrame()
                    temp['movieId'] = movies_metadata['movieId'].iloc[movie_indices]
                    temp['weights'] = weight_movies
                    temp2['movieId'] = unrated_movies 
                    merged_rating_df = pd.merge(temp, temp2, how='inner', left_on=['movieId'], right_on = ['movieId'])
                    merged_rating_df = pd.merge(train_df.loc[(train_df['userId'] == users[i])], merged_rating_df , how='inner', left_on=['movieId'], right_on = ['movieId'])
                    merged_rating_df["weights*rating"] = merged_rating_df["weights"] * merged_rating_df["rating"]
                    weights = merged_rating_df['weights'].values
                    weights_rating = merged_rating_df['weights*rating'].values
                    prediction = sum(weights_rating)/(sum(weights))
                    test_df1.loc[(test_df1['userId'] == users[i]) & (test_df1['movieId'] == movie),'prediction'] = prediction     
    end1 = time.time()
    print('User : ', users[i])
    print(end1 - start1)  
end = time.time()
print(end - start)       

User :  1
0.3341064453125
User :  7
1.7453362941741943
User :  31
1.3872835636138916
User :  32
0.9155523777008057
User :  36
2.1273505687713623
User :  39
0.795203685760498
User :  73
29.39163112640381
User :  88
3.7110800743103027
User :  96
1.0053327083587646
User :  110
1.6366286277770996
User :  111
4.717014789581299
User :  150
6.758333683013916
User :  161
1.337465524673462
User :  165
7.0958569049835205
User :  186
0.4911665916442871
User :  242
5.767437934875488
User :  254
1.9498629570007324
User :  288
1.0056207180023193
User :  310
0.14819765090942383
User :  311
14.83066725730896
User :  325
0.22678875923156738
User :  338
0.8467342853546143
User :  341
0.17655706405639648
User :  386
1.693955659866333
User :  394
5.278679847717285
User :  452
22.457972288131714
User :  457
10.753352165222168
User :  461
10.39245080947876
User :  468
19.4545841217041
User :  485
0.2655515670776367
User :  487
2.257037878036499
User :  496
1.8378558158874512
User :  511
0.35289692878723145


User :  226
0.4268310070037842
User :  304
1.6314184665679932
User :  381
1.524564266204834
User :  412
3.222539186477661
User :  480
5.355878114700317
User :  557
0.8901209831237793
User :  595
2.259112596511841
User :  611
0.34793639183044434
User :  656
1.7889103889465332
User :  75
1.9211914539337158
User :  157
4.377346754074097
User :  167
0.20149803161621094
User :  199
6.0097596645355225
User :  355
4.326808452606201
User :  395
0.1825122833251953
User :  482
0.6869819164276123
User :  544
3.837790012359619
User :  599
2.7023022174835205
User :  626
2.0296173095703125
User :  68
1.719451904296875
User :  282
1.744386911392212
User :  76
0.1619429588317871
User :  83
2.346726179122925
User :  125
2.9812064170837402
User :  134
5.129852056503296
User :  178
1.8101861476898193
User :  196
1.3057403564453125
User :  222
1.2526462078094482
User :  315
0.7480447292327881
User :  352
1.0296149253845215
User :  398
0.682180643081665
User :  424
0.30701327323913574
User :  519
2.1622307

User :  652
3.8444204330444336
User :  323
0.20445942878723145
User :  462
0.6582341194152832
User :  246
0.23259758949279785
User :  367
3.0203349590301514
User :  180
0.21342825889587402
User :  218
0.5362935066223145
User :  230
1.336425542831421
User :  612
0.8487281799316406
User :  60
0.7500309944152832
User :  348
0.8746597766876221
User :  393
0.9424886703491211
User :  644
0.5877904891967773
User :  12
0.807525634765625
User :  189
2.519655704498291
User :  215
0.7617199420928955
User :  368
0.19922327995300293
User :  645
0.345228910446167
User :  515
0.3831312656402588
User :  107
0.3680109977722168
User :  112
0.16156506538391113
User :  181
0.28672099113464355
User :  208
0.5659995079040527
User :  274
0.45572376251220703
User :  278
0.67618727684021
User :  334
0.3803689479827881
User :  445
0.1466081142425537
User :  506
0.2363719940185547
User :  524
0.8128266334533691
User :  583
0.18051838874816895
User :  609
2.013615608215332
User :  668
0.13463377952575684
User :  

In [29]:
global_mean_rating = np.average(train_df.rating.values)

In [30]:
global_mean_rating

3.5356324435981024

In [31]:
test_df1 = test_df1.fillna(global_mean_rating)

### Performance Evaluation on the training data

In [32]:
# The actual rating of the movies by the user
y_test = test_df1['rating'].values
# The predicted rating of the movies
y_test_pred = test_df1['prediction'].values

In [33]:
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.metrics import mean_absolute_error
# Root Mean Square Error of the model
rms = round(sqrt(mean_squared_error(y_test, y_test_pred )), 2)
print('Root Mean Square Error:',rms)
# Mean Absolute Error of the model
mse = round(mean_absolute_error(y_test, y_test_pred), 2)
print('Mean Absolute Error:',mse)

Root Mean Square Error: 1.06
Mean Absolute Error: 0.85


In [34]:
test_df1.to_csv("train_predict_content_df.csv", index=False)